In [1]:
# https://www.aanda.org/articles/aa/pdf/2006/45/aa5897-06.pdf

In [2]:
# http://www.iausofa.org/sofa_ast_c.pdf
# https://www.iers.org/SharedDocs/Publikationen/EN/IERS/Publications/tn/TechnNote32/tn32.pdf;jsessionid=E7DE9BECBE5D3FD0BE0D6F4CE81DDE30.live1?__blob=publicationFile&v=1
# https://www.iausofa.org/sofa_pn_c.pdf

# Polar motion
# http://hpiers.obspm.fr/eop-pc/index.php?index=pm&lang=ru

In [3]:
import astropy
import math
import numpy as np
from astropy.time import Time
from astropy.coordinates import Angle
import numpy.polynomial.polynomial as poly

In [4]:
import erfa

In [5]:
def rot1(ang):
    return [[1,0,0], 
            [0, np.cos(ang), np.sin(ang)], 
            [0, -np.sin(ang), np.cos(ang)]
           ]

def rot2(ang):
    return [ 
            [np.cos(ang), 0, -np.sin(ang)],
            [0, 1, 0],
            [np.sin(ang), 0, np.cos(ang)]
           ]

def rot3(ang):
    return [[np.cos(ang), np.sin(ang), 0], 
            [-np.sin(ang), np.cos(ang), 0],
            [0, 0, 1]
           ]

In [6]:
j2000 = Time("J2000", format='jyear_str', scale="tt")

In [7]:
thist = Time("2006-8-16T00:36:37.5", scale='utc')
print(thist)

2006-08-16T00:36:37.500


In [8]:
thist.tt.jd

2453963.5261884723

In [9]:
thist.to_value(format='jyear')

2006.621561763252

In [10]:
utc1, utc2 = erfa.dtf2d('UTC', 2006,8,16,0,36,37.5)
tai1, tai2 = erfa.utctai(utc1, utc2)
tt1, tt2 = erfa.taitt(tai1, tai2)
gmst = erfa.gmst06(thist.ut1.jd, 0, thist.tt.jd, 0)
gmst = Angle(gmst, unit='rad')
print(gmst.hms)

hms_tuple(h=22.0, m=13.0, s=43.39808020008434)


In [11]:
# Earth Rotation Angle
def era(thist):
    tu = thist.ut1.jd - 2451545.0
    a = 0.7790572732640 + 1.00273781191135448 * tu
    a = a % 1
    return Angle(2 * math.pi * a, unit='rad')

In [12]:
era_ang = era(thist)
era_ang.value

5.817985023917661

In [13]:
erfa.era00(thist.ut1.jd, 0)

5.817985023917316

In [14]:
s = erfa.s06a(tt1, tt2) # CIO locator

In [15]:
mcio = erfa.c2i06a(tt1, tt2)
Xx, Yy, Ss = erfa.xys06a(tt1, tt2)

In [16]:
rmat = rot3(era_ang) @ mcio
rmat

array([[ 8.93731561e-01, -4.48601812e-01, -5.58340872e-04],
       [ 4.48601693e-01,  8.93731734e-01, -3.29709875e-04],
       [ 6.46915403e-04,  4.41994616e-05,  9.99999790e-01]])

In [17]:
# TIO locator
sp = erfa.sp00(tt1, tt2)

In [18]:
# http://hpiers.obspm.fr/eoppc/eop/eopc01/filtered-pole.tab
# Polar motion
# values in arcsec
xp = Angle(0.063424, unit='arcsec')         
yp = Angle(0.345493 , unit='arcsec')

In [19]:
rpm = erfa.pom00(xp.rad, yp.rad, sp)
rpm

array([[ 1.00000000e+00, -1.50880563e-11,  3.07488229e-07],
       [ 1.56030982e-11,  1.00000000e+00, -1.67499733e-06],
       [-3.07488229e-07,  1.67499733e-06,  1.00000000e+00]])

In [20]:
rglob = rpm @ rot3(era_ang) @ mcio
rglob

array([[ 8.93731561e-01, -4.48601812e-01, -5.58033383e-04],
       [ 4.48601692e-01,  8.93731734e-01, -3.31384872e-04],
       [ 6.47391998e-04,  4.58343996e-05,  9.99999789e-01]])

In [21]:
dec = Angle("8d52m06s")
ra = Angle("19h50m47s")

In [22]:
lon =  Angle("-3d41m31s")
phi = Angle("40d25m")
hsite = 650

In [23]:
v1 = (np.cos(dec)*np.cos(ra), np.cos(dec)*np.sin(ra), np.sin(dec))

In [24]:
astrom, eo = erfa.apci13(thist.tdb.jd, 0)
# equivalente a mcio
bpn = astrom[6]

In [25]:
# Lo mismo
p = erfa.s2c(ra.rad, dec.rad)
q = erfa.rxp(bpn, p)
qq = erfa.c2s(q)
qq = (erfa.anp(qq[0]), qq[1])

In [26]:
# Lo mismo
q2 = mcio @ v1
qq2 = erfa.c2s(q2)
qq2 = (erfa.anp(qq2[0]), qq2[1])

In [27]:
# No incluye ERA
# Lo mismo
res = erfa.atci13(ra.rad, dec.rad, 0,0,0,0, thist.tdb.jd, 0)

In [28]:
ra22 = Angle(res[0], unit='rad')
dec22 = Angle(res[1], unit='rad')

In [29]:
ra22.rad, qq[0], qq2[0]

(5.195766052940791, 5.19567978925342, 5.19567978925342)

In [30]:
dec22.rad, qq[1], qq2[1]

(0.15507989903263658, 0.15504312477718582, 0.15504312477718993)

In [31]:
astrom2 = erfa.apio13(thist.utc.jd,0, thist.delta_ut1_utc, 
                      lon.rad, phi.rad, hsite,
                      xp.rad, yp.rad,
                      0,0,0,0.5) # refraccion

In [32]:
astrom2[7], lon.rad + sp

(-0.06443658637135649, -0.06443658637135649)

In [33]:
era_local = era_ang + lon + Angle(sp, unit='rad')

In [34]:
era_local.rad, astrom2[14], erfa.era00(thist.ut1.jd, 0) + astrom2[7]

(5.753548437546304, 5.753548437772936, 5.75354843754596)

In [35]:
h = (ra22 - era_local)
v3 = erfa.s2c(h.rad, dec22.rad)

In [36]:
v3

array([ 0.83824914, -0.52295391,  0.15445904])

In [37]:
# rot to horizontal coords
v4x = np.sin(phi)*v3[0] - np.cos(phi)*v3[2]
v4y = v3[1]
v4z = np.cos(phi)*v3[0] + np.sin(phi)*v3[2]

In [38]:
altobs = Angle(180, unit='deg') + np.arctan2(v4y,-v4x)

In [39]:
altobs

<Angle 50.84190652 deg>

In [40]:
aobs = Angle(90, unit='deg') - np.arctan2(np.sqrt(v4x**2+v4y**2), v4z);

In [41]:
# No refraction
altobs.deg, aobs.deg

(50.84190652452955, 47.59045149434478)

In [42]:
H = -h

In [43]:
dec22.deg, H.hourangle

(8.885423701885014, 2.1305717683092515)

In [46]:
print(H.to_string(unit='degree'))

31d57m30.8755s
